# "Implémentez un modèle de scoring"
_Cleaning the data | Exploratory Data Analysis_


References used for this notebook:

- "https://www.kaggle.com/gpreda/home-credit-default-risk-extensive-eda"
- 

## 0 Preliminaries

### 0.0 Importing Packages and Modules

Checking whether the notebook is on Colab or PC

In [5]:
import sys
is_colab = 'google.colab' in sys.modules
is_colab, sys.executable

(True, '/usr/bin/python3')

Mounting my Drive if on Colab

In [6]:
if is_colab==True:
    from google.colab import files, output, drive
    drive.mount('/gdrive')
    %cd /gdrive
    print("You're on Google Colab")
else:
    print("You're on a PC")

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive
You're on Google Colab


Installations and importations required in the virtual environment.

In [7]:
import os
os.chdir('/gdrive/My Drive/--DATA SCIENCE/PROJET7/NOTEBOOKS')
# !pip install -r requirements.txt

In [8]:
from P7_functions import *

Installations (creating the requirements file)

In [9]:
# !pip freeze > requirements_merging.txt

Importation of modules and packages. 

In [10]:
import io
import string
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.facecolor']='w'

In [11]:
import missingno as msno

Setting pandas display options.

In [12]:
dictPdSettings = {'display.max_rows': 500, 'display.width': 100,
                  'display.max_colwidth': 100,
                  'display.float_format': lambda x: '%.2f' % x}
for k,v in dictPdSettings.items():
  pd.set_option(k,v)

### 0.1 Importing the datasets

In [13]:
path_data = '/gdrive/My Drive/--DATA SCIENCE/PROJET7/DATA/'

In [14]:
# get the name of all the .csv files in the 'DATA' directory

li_files = []
for _, _, n_files in os.walk(path_data):
    li_files = n_files
li_files

['HomeCredit_columns_description.csv',
 'credit_card_balance.csv',
 'application_test.csv',
 'bureau.csv',
 'sample_submission.csv',
 'previous_application.csv',
 'application_train.csv',
 'bureau_balance.csv',
 'POS_CASH_balance.csv',
 'installments_payments.csv']

In [15]:
li_key_names = ['home_cred_desc','ccard_bal','app_test','bur', 'samp_subm',
                'prev_app','app_train','bur_bal','pos_cash_bal','install_pay']
dict_df = {}

In [16]:
for k, n_file in zip(li_key_names, li_files):
    dict_df[k] = pd.read_csv(path_data+n_file, encoding = "ISO-8859-1")

In [17]:
dict_df.keys()

dict_keys(['home_cred_desc', 'ccard_bal', 'app_test', 'bur', 'samp_subm', 'prev_app', 'app_train', 'bur_bal', 'pos_cash_bal', 'install_pay'])

## 1 Overview

### 1.0 Size

In [18]:
# Have a overview of the content

for k, df in dict_df.items():
    display(f"{k}: {df.shape}")

'home_cred_desc: (219, 5)'

'ccard_bal: (3840312, 23)'

'app_test: (48744, 121)'

'bur: (1716428, 17)'

'samp_subm: (48744, 2)'

'prev_app: (1670214, 37)'

'app_train: (307511, 122)'

'bur_bal: (27299925, 3)'

'pos_cash_bal: (10001358, 8)'

'install_pay: (13605401, 8)'

In [19]:
# application_train
# application_test
# bureau
# bureau_balance
# credit_card_balance
# credit_card_balance
# installments_payments
# previous_application
# POS_CASH_balance

### 1.1 Missing values

In [21]:
def missing_data(data):
    
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100)\
                                .sort_values(ascending = False)
    return pd.concat([total, percent],
                     axis=1,
                     keys=['Total', 'Percent'])

In [25]:
# Have a overview of the missing values

for k, df in dict_df.items():
    print(k)
    display(missing_data(df).head(10))

home_cred_desc


,Total,Percent
Special,133,60.73
Description,0,0.00
Row,0,0.00
Table,0,0.00
Unnamed: 0,0,0.00


ccard_bal


,Total,Percent
AMT_PAYMENT_CURRENT,767988,20.00
AMT_DRAWINGS_OTHER_CURRENT,749816,19.52
CNT_DRAWINGS_POS_CURRENT,749816,19.52
CNT_DRAWINGS_OTHER_CURRENT,749816,19.52
CNT_DRAWINGS_ATM_CURRENT,749816,19.52
AMT_DRAWINGS_ATM_CURRENT,749816,19.52
AMT_DRAWINGS_POS_CURRENT,749816,19.52
CNT_INSTALMENT_MATURE_CUM,305236,7.95
AMT_INST_MIN_REGULARITY,305236,7.95
SK_DPD_DEF,0,0.00


app_test


...

,Total,Percent
STATUS,0,0.00
MONTHS_BALANCE,0,0.00
SK_ID_BUREAU,0,0.00


pos_cash_bal


,Total,Percent
CNT_INSTALMENT_FUTURE,26087,0.26
CNT_INSTALMENT,26071,0.26
SK_DPD_DEF,0,0.00
SK_DPD,0,0.00
NAME_CONTRACT_STATUS,0,0.00
MONTHS_BALANCE,0,0.00
SK_ID_CURR,0,0.00
SK_ID_PREV,0,0.00


install_pay


,Total,Percent
AMT_PAYMENT,2905,0.02
DAYS_ENTRY_PAYMENT,2905,0.02
AMT_INSTALMENT,0,0.00
DAYS_INSTALMENT,0,0.00
NUM_INSTALMENT_NUMBER,0,0.00
NUM_INSTALMENT_VERSION,0,0.00
SK_ID_CURR,0,0.00
SK_ID_PREV,0,0.00
